In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime

# Importing 3 dataframes

In [2]:
df_parcel = pd.read_csv('../Data/raw/EXTR_Parcel.csv', encoding = "ISO-8859-1")

In [3]:
df_res = pd.read_csv('../Data/raw/EXTR_ResBldg.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_real = pd.read_csv('../Data/raw/EXTR_RPSale.csv', encoding = "ISO-8859-1")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Filtering Real Sales Dataframe

In [5]:
#set to 
df_real_home = df_real.loc[(df_real.PropertyType.isin([2,3,11])) & (df_real.SalePrice > 0)]

In [6]:
#Creating Column to combine Major and minor called UniqueID 
df_real_home['UniqueID'] = (df_real_home['Major'].astype(str)+ df_real_home['Minor'].astype(str)).astype(int)

<ipython-input-6-acffeaa3d2a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_home['UniqueID'] = (df_real_home['Major'].astype(str)+ df_real_home['Minor'].astype(str)).astype(int)


In [7]:
df_real_home['DocumentDate'] = pd.to_datetime(df_real_home['DocumentDate'], format="%m/%d/%Y")

<ipython-input-7-4a2c7d9ac08f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_home['DocumentDate'] = pd.to_datetime(df_real_home['DocumentDate'], format="%m/%d/%Y")


In [8]:
#dropping duplicated values of unique id and keeping the most recent sale date
df_real_home = df_real_home.sort_values('DocumentDate').drop_duplicates('UniqueID', keep='last')

# Filtering Parcel and Resident dataframe

In [9]:
df_res['UniqueID'] = (df_res['Major'].astype(str)+ df_res['Minor'].astype(str)).astype(int)
df_parcel['UniqueID'] = (df_parcel['Major'].astype(str)+ df_parcel['Minor'].astype(str)).astype(int)

In [10]:
df_parcel = df_parcel[df_parcel.PropType == 'R']

In [11]:
df_parcel = df_parcel.drop_duplicates(subset = 'UniqueID', keep = 'first')

In [12]:
df_res = df_res.drop_duplicates(subset = 'UniqueID', keep = 'first')

# Making uniqueID the index for all the dataframes

In [13]:
df_parcel.set_index('UniqueID', drop = True, inplace = True)

In [14]:
df_res.set_index('UniqueID', drop = True, inplace = True)

In [15]:
df_real_home.set_index('UniqueID', drop = True, inplace = True)

# Concatenating the dateframes 

In [20]:
test_df = pd.concat([df_real_home, df_parcel.reindex(df_real_home.index)], axis=1)

In [22]:
df_combined = pd.concat([test_df, df_res.reindex(test_df.index)], axis=1)

In [32]:
df_combined = df_combined.loc[:,~df_combined.columns.duplicated()]

In [57]:
df_combined.Area.value_counts()

69.0    8983
35.0    8324
6.0     7872
37.0    7695
56.0    7588
        ... 
94.0    2736
66.0    2638
57.0    2166
63.0    1338
90.0     677
Name: Area, Length: 88, dtype: int64

In [34]:
df_combined.to_json('../Data/combined.json')

In [58]:
df_final = df_combined[['Major', 'Minor', 'DocumentDate','SalePrice', 'PropertyType', 'PrincipalUse', 'SaleReason', 'PropertyClass', 'SaleWarning', 'Area', 'SqFtLot', 'LakeWashington',
 'LakeSammamish','SmallLakeRiverCreek','OtherView','WfntLocation','WfntFootage','WfntBank','WfntPoorQuality','WfntRestrictedAccess','WfntAccessRights','WfntProximityInfluence','TidelandShoreland', 'TrafficNoise',
 'AirportNoise','PowerLines','OtherNuisances', 'SqFtTotLiving','SqFtDeck','SqFtOpenPorch', 'Condition' ]]

In [53]:
df_combined.PropType.value_counts()

R    440326
Name: PropType, dtype: int64

In [62]:
df_final.head()

,Major,Minor,DocumentDate,SalePrice,PropertyType,PrincipalUse,SaleReason,PropertyClass,SaleWarning,Area,...,WfntProximityInfluence,TidelandShoreland,TrafficNoise,AirportNoise,PowerLines,OtherNuisances,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,Condition
UniqueID,,,,,,,,,,,,,,,,,,,,,
7186800030,718680,0030,1954-03-11,600,3,6,18,8,32 51,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222470151,222470,151,1959-10-10,12250,3,0,1,8,18,2.0,...,N,0.0,1.0,0.0,N,N,2090.0,0.0,0.0,4.0
46700477,46700,477,1960-01-01,2500,3,0,18,2,22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503180251,503180,251,1960-09-27,2300,3,0,1,8,25,100.0,...,N,2.0,0.0,0.0,N,N,1140.0,810.0,40.0,3.0
55893045,558930,45,1962-10-03,7200,3,6,1,8,,3.0,...,N,0.0,0.0,0.0,N,N,480.0,0.0,0.0,3.0


In [71]:
df_final['year'] = pd.DatetimeIndex(df_final['DocumentDate']).year

<ipython-input-71-b02a7a58dda0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['year'] = pd.DatetimeIndex(df_final['DocumentDate']).year


In [72]:
df_final.head()

,Major,Minor,DocumentDate,SalePrice,PropertyType,PrincipalUse,SaleReason,PropertyClass,SaleWarning,Area,...,TidelandShoreland,TrafficNoise,AirportNoise,PowerLines,OtherNuisances,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,Condition,year
UniqueID,,,,,,,,,,,,,,,,,,,,,
7186800030,718680,0030,1954-03-11,600,3,6,18,8,32 51,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1954
222470151,222470,151,1959-10-10,12250,3,0,1,8,18,2.0,...,0.0,1.0,0.0,N,N,2090.0,0.0,0.0,4.0,1959
46700477,46700,477,1960-01-01,2500,3,0,18,2,22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960
503180251,503180,251,1960-09-27,2300,3,0,1,8,25,100.0,...,2.0,0.0,0.0,N,N,1140.0,810.0,40.0,3.0,1960
55893045,558930,45,1962-10-03,7200,3,6,1,8,,3.0,...,0.0,0.0,0.0,N,N,480.0,0.0,0.0,3.0,1962


In [75]:
df_2019final = df_final[df_final.year == 2019]

In [76]:
df_2019final.to_json('../Data/combined.json')